In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). \
As of 12/23 I'm realzing that I'll need to start with a complete list of all ZIP codes (from USPS) before I start this so I can get a good sense of completeness. 

Below, I'm reading in that full list of ZIP codes

In [3]:
ZIP_to_CD_df = pd.read_excel('Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

In [4]:
col_types = {
    'DELIVERY ZIPCODE': str
}

FULL_ZIPS = pd.read_excel('Data/Full_Zips.xlsx', dtype=col_types)

print(len(FULL_ZIPS))

FULL_ZIPS = FULL_ZIPS[['DELIVERY ZIPCODE','PHYSICAL STATE','TYPE']]

FULL_ZIPS = FULL_ZIPS.rename(columns={
    'DELIVERY ZIPCODE': 'ZIP',
    'PHYSICAL STATE': 'USPS_STATE'
})

#removing ZIP codes that aren't in US main
FULL_ZIPS = FULL_ZIPS.drop_duplicates(subset='ZIP', keep='first')
FULL_ZIPS = FULL_ZIPS[pd.notna(FULL_ZIPS['USPS_STATE'])].reset_index(drop=True)
exclude_states = ["VI", "PR", "MH", "MP", "FM", "GU"]
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['USPS_STATE'].isin(exclude_states)].reset_index(drop=True)
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['TYPE'].isin(["Other", "Unique"])].reset_index(drop=True)

print(len(FULL_ZIPS))
FULL_ZIPS.head()

49649
37714


/Users/chrissoria/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ZIP,USPS_STATE,TYPE
0,01001,MA,Standard/PO BOX
1,01002,MA,Standard/PO BOX
2,01004,MA,Standard/PO BOX
3,01005,MA,Standard/PO BOX
4,01007,MA,Standard/PO BOX


After merging, I find that many of the Alaska ZIP codes aren't getting matched. Since there's only one CD in Alaska, I'll fill in any Alaska state ZIP with 0200. \
This data will not match PO boxes. Most non matches are coming from PO boxes.

In [5]:
ZIP_to_CD_df = pd.merge(FULL_ZIPS, ZIP_to_CD_df, on = "ZIP", how = "left")

ZIP_to_CD_df['CD'] = ZIP_to_CD_df.apply(lambda row: "0200" if row['USPS_STATE'] == "AK" else row['CD'], axis = 1)
ZIP_to_CD_df['CD_TOT_RATIO'] = ZIP_to_CD_df.apply(lambda row: 1.0 if row['USPS_STATE'] == "AK" else row['CD_TOT_RATIO'], axis = 1)

ZIP_to_CD_df['CD'] = ZIP_to_CD_df.apply(lambda row: "4212" if row['ZIP'] == "15253" else row['CD'],
                                       axis=1)

ZIP_to_CD_df

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO
0,01001,MA,Standard/PO BOX,2501,1.000000,1.0,1.0,1.000000
1,01002,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
2,01004,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
3,01005,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
4,01007,MA,Standard/PO BOX,2502,0.998118,1.0,1.0,0.998217
...,...,...,...,...,...,...,...,...
44276,99925,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44277,99926,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44278,99927,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44279,99928,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000


For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially \
This will NOT include DC simply because DC doesn't have a CD \
IN A FUTURE VERSION, I WILL HAVE TO START WITH A LIST OF ZIP CODES THAT INCLUDE DISTRICT OF COLUMBIA'S (WHICH I  HAVE ABOVE)

In [6]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/ZIP_COUNTY_062020_HUD.xlsx', dtype=col_types)
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)

ZIP_to_County_df = pd.merge(ZIP_to_CD_df, ZIP_to_County_df, how ='outer', on = "ZIP")

# Sort the DataFrame by RES_RATIO in descending order
ZIP_to_County_df = ZIP_to_County_df.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_County_df = ZIP_to_County_df.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

ZIP_FEATURES = pd.merge(ZIP_to_CD_df, ZIP_to_County_df)

I'm realizing I'll have to merge the school district data onto ZIP_to_County_df (give that there's some zip codes that don't have a school district associated) \
Yet, the issue of some ZIP codes falling through the cracks will still be there. \
I'll need something that goes from county and matches to all ZIP codes within OR \
Something that matches ZIP codes directly to the nearest school district (IDEAL). \
For now (12/26/23) this variable is not complete. 

Instead, I'll match the ZIP code to a census tract THEN match tract onto school district

In [7]:
col_types = {
    'TRACT': str,
    'ZIP': str
}

ZIP_to_Tract = pd.read_excel('data/ZIP_TRACT_062020.xlsx', dtype=col_types)

ZIP_to_Tract = ZIP_to_Tract.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_Tract = ZIP_to_Tract.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_Tract.reset_index(drop=True, inplace=True)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, ZIP_to_Tract, on = "ZIP", how = "left")

ZIP_FEATURES

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY,COUNTY_RES_RATIO,COUNTY_BUS_RATIO,COUNTY_OTH_RATIO,COUNTY_TOT_RATIO,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,01001,MA,Standard/PO BOX,2501,1.000000,1.0,1.0,1.000000,25013,1.000000,1.000000,1.000000,1.000000,25013813209,0.404431,0.442809,0.250000,0.403828
1,01002,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25015,0.988689,0.998889,0.997312,0.989909,25015820500,0.280456,0.602222,0.548387,0.318705
2,01004,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25015,1.000000,1.000000,1.000000,1.000000,25015820500,1.000000,1.000000,1.000000,1.000000
3,01005,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25027,1.000000,1.000000,1.000000,1.000000,25027722100,0.992387,1.000000,0.985401,0.992145
4,01007,MA,Standard/PO BOX,2502,0.998118,1.0,1.0,0.998217,25015,1.000000,1.000000,1.000000,1.000000,25015820203,0.591843,0.619883,0.800000,0.598277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37709,99925,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198000200,0.000000,0.000000,1.000000,1.000000
37710,99926,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198940100,0.000000,0.000000,1.000000,1.000000
37711,99927,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198000100,0.000000,0.000000,1.000000,1.000000
37712,99928,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02130,0.000000,0.000000,1.000000,1.000000,02130000100,0.000000,0.000000,1.000000,1.000000


Now, let's match this tract to a school district. \
After merging this file, I find that there are still a large chunk of rows without a school district \
For these rows, I'll use a cruder approximation of school district at the county-level (which might be too broad)

In [8]:
col_types = {
    'LEAID': str,
    'TRACT': str
}

school_to_tract = pd.read_excel('data/grf20_lea_tract.xlsx', dtype=col_types)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, school_to_tract, on = "TRACT", how = "left")

Below, I'll read in the county-district level data

In [9]:
col_types = {
    'LEAID': str,
    'STCOUNTY': str
}

County_to_District = pd.read_excel('data/grf20_lea_county.xlsx', dtype=col_types)
County_to_District = County_to_District.sort_values(by='LANDAREA', ascending=False)
County_to_District = County_to_District.drop_duplicates(subset='STCOUNTY', keep='first')

County_to_District = County_to_District.rename(columns={"STCOUNTY": "COUNTY"})

County_to_District = County_to_District[['NAME_LEA20','COUNTY','LEAID']]

ZIP_FEATURES = pd.merge(ZIP_FEATURES, County_to_District, on = "COUNTY", how = "left")

Next, I'll clean up the data so that we keep only the desired columns, differentiate between the census tract (higher resolution) school district match and the county-level (lower resolution) match. Lastly, I'll create one column for school district based on both of these types of matches

In [10]:
ZIP_FEATURES.head()

columns_to_keep = ['ZIP','USPS_STATE','CD','COUNTY','TRACT','LEAID_x','LEAID_y','NAME_LEA20_x','NAME_LEA20_y']

ZIP_FEATURES = ZIP_FEATURES[columns_to_keep]

ZIP_FEATURES = ZIP_FEATURES.rename(columns = {
                                   'NAME_LEA20_x': 'TRACT_SCH_DISTRICT',
                                   'NAME_LEA20_y': 'COUNTY_SCH_DISTRICT',
                                   'LEAID_x': 'TRACT_LEAID',
                                   'LEAID_y': 'COUNTY_LEAID'})

ZIP_FEATURES['SCHOOL_DISTRICT'] = ZIP_FEATURES.apply(lambda row: row['COUNTY_SCH_DISTRICT'] if pd.isna(row['TRACT_SCH_DISTRICT']) else row['TRACT_SCH_DISTRICT'], axis = 1)
ZIP_FEATURES['LEAID'] = ZIP_FEATURES.apply(lambda row: row['COUNTY_LEAID'] if pd.isna(row['TRACT_LEAID']) else row['TRACT_LEAID'], axis = 1)


ZIP_FEATURES.head()

,ZIP,USPS_STATE,CD,COUNTY,TRACT,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID
0,01001,MA,2501,25013,25013813209,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800
1,01002,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890
2,01002,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920
3,01004,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890
4,01004,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920


NOTE: The school closure data provides best coverage for months after september 2020 to May 2021

In [11]:
schools_closures = pd.read_csv('data/District_Monthly_Shares_03.08.23.csv',dtype=str)

columns_to_keep = ['NCESDistrictID','month','share_inperson','DistrictName']

schools_closures = schools_closures[columns_to_keep]

schools_closures['LEAID'] = schools_closures['NCESDistrictID'].apply(lambda x: '00' + x if len(x) == 5 else x)

schools_closures['NCESDistrictID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

#for mid 2020 (taking two months in case the first is missing August and September)

schools_closures_m2020 = schools_closures[schools_closures['month'] == "2020m8"].reset_index(drop=True)

schools_closures_m2020 = schools_closures_m2020.rename(columns={
    'share_inperson': 'august20_school_share_inperson'
})

schools_closures_m2020 = schools_closures_m2020[['LEAID','august20_school_share_inperson','DistrictName']]

#now mnth 9
schools_closures_m2020_2 = schools_closures[schools_closures['month'] == "2020m9"].reset_index(drop=True)

schools_closures_m2020_2 = schools_closures_m2020_2.rename(columns={
    'share_inperson': 'september20_school_share_inperson'
})

schools_closures_m2020_2 = schools_closures_m2020_2[['LEAID','september20_school_share_inperson']]

#for month 10 (aggressive capture)

schools_closures_m2020_3 = schools_closures[schools_closures['month'] == "2020m10"].reset_index(drop=True)

schools_closures_m2020_3 = schools_closures_m2020_3.rename(columns={
    'share_inperson': 'october20_school_share_inperson'
})

schools_closures_m2020_3 = schools_closures_m2020_3[['LEAID','october20_school_share_inperson']]

#for late 2020

schools_closures_l2020 = schools_closures[schools_closures['month'] == "2020m12"].reset_index(drop=True)

schools_closures_l2020 = schools_closures_l2020.rename(columns={
    'share_inperson': 'december20_school_share_inperson'
})

schools_closures_l2020 = schools_closures_l2020[['LEAID','december20_school_share_inperson']]


schools_closures_l2020_2 = schools_closures[schools_closures['month'] == "2020m11"].reset_index(drop=True)

schools_closures_l2020_2 = schools_closures_l2020_2.rename(columns={
    'share_inperson': 'november20_school_share_inperson'
})

schools_closures_l2020_2 = schools_closures_l2020_2[['LEAID','november20_school_share_inperson']]


#for mid 2021

schools_closures_m2021 = schools_closures[schools_closures['month'] == "2021m3"].reset_index(drop=True)

schools_closures_m2021 = schools_closures_m2021.rename(columns={
    'share_inperson': 'march21_school_share_inperson'
})

schools_closures_m2021 = schools_closures_m2021[['LEAID','march21_school_share_inperson']]


schools_closures_m2021_2 = schools_closures[schools_closures['month'] == "2021m2"].reset_index(drop=True)

schools_closures_m2021_2 = schools_closures_m2021_2.rename(columns={
    'share_inperson': 'feb21_school_share_inperson'
})

schools_closures_m2021_2 = schools_closures_m2021_2[['LEAID','feb21_school_share_inperson']]

#later mid 2021

schools_closures_lm2021 = schools_closures[schools_closures['month'] == "2021m4"].reset_index(drop=True)

schools_closures_lm2021 = schools_closures_lm2021.rename(columns={
    'share_inperson': 'april21_school_share_inperson'
})

schools_closures_lm2021 = schools_closures_lm2021[['LEAID','april21_school_share_inperson']]


schools_closures_lm2021_2 = schools_closures[schools_closures['month'] == "2021m5"].reset_index(drop=True)

schools_closures_lm2021_2 = schools_closures_lm2021_2.rename(columns={
    'share_inperson': 'may21_school_share_inperson'
})

schools_closures_lm2021_2 = schools_closures_lm2021_2[['LEAID','may21_school_share_inperson']]


Early fall = Aug-Oct
Fall 2020 = Nov-Dec
Spring 2021 = Feb-Mar

In [12]:
schools_closures = schools_closures[~schools_closures['LEAID'].duplicated()]

schools_closures = pd.merge(schools_closures, schools_closures_m2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2020_3, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_lm2021_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_lm2021, how = 'left', on = 'LEAID')

#aug, sept, oct (2020)
schools_closures['earlyfall2020_school_closures'] = schools_closures.apply(
    lambda row: row['september20_school_share_inperson'] if pd.isna(row['august20_school_share_inperson']) else row['august20_school_share_inperson'], 
    axis=1
)
schools_closures['earlyfall2020_school_closures'] = schools_closures.apply(
    lambda row: row['october20_school_share_inperson'] if pd.isna(row['earlyfall2020_school_closures']) else row['earlyfall2020_school_closures'],
    axis=1
)

#nov, dec (2020)
schools_closures['fall2020_school_closures'] = schools_closures.apply(
    lambda row: row['november20_school_share_inperson'] if pd.isna(row['december20_school_share_inperson']) else row['december20_school_share_inperson'],
    axis=1
)

#feb, march (2021)
schools_closures['earlyspring2021_school_closures'] = schools_closures.apply(
    lambda row: row['feb21_school_share_inperson'] if pd.isna(row['march21_school_share_inperson']) else row['march21_school_share_inperson'], 
    axis=1
)

#april, may (2021)

schools_closures['spring2021_school_closures'] = schools_closures.apply(
    lambda row: row['april21_school_share_inperson'] if pd.isna(row['may21_school_share_inperson']) else row['may21_school_share_inperson'], 
    axis=1
)

schools_closures = schools_closures[['LEAID','earlyfall2020_school_closures','fall2020_school_closures',
                                     'earlyspring2021_school_closures','spring2021_school_closures',]]
schools_closures.to_csv('data/school_closure_merged.csv')

schools_closures

,LEAID,earlyfall2020_school_closures,fall2020_school_closures,earlyspring2021_school_closures,spring2021_school_closures
0,200001,.0032779209,.0029382957,.0032122559,.85860348
1,200003,0,0,.03125,.36306652
2,200004,0,0,0,0
3,200005,1,0,0,1
4,200006,1,1,1,1
...,...,...,...,...,...
14962,5605762,1,1,1,1
14963,5605820,1,1,1,1
14964,5605830,0,0,0,0
14965,5606090,1,1,1,1


let's take a look at coverage. Based on these missing counts, it looks like the best coverage available is for fall 2020, with early spring 2021 and late spring 2021 having a higher percentage of missingness (almost 10 percent of rows contain missing data)

In [13]:
na_counts = schools_closures[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                              'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().sum()

na_percentage = schools_closures[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                                  'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().mean() * 100


print(na_counts)
print(na_percentage)

LEAID                                 0
earlyfall2020_school_closures        36
fall2020_school_closures             36
earlyspring2021_school_closures    1307
spring2021_school_closures         1452
dtype: int64
LEAID                              0.000000
earlyfall2020_school_closures      0.240529
fall2020_school_closures           0.240529
earlyspring2021_school_closures    8.732545
spring2021_school_closures         9.701343
dtype: float64


In [14]:
ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].astype(str)

ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, schools_closures, how = 'left', on = "LEAID")

In terms of ZIP codes, this translates to big percentages. Over 20% of zip codes do not have data on school closures for the spring 2021 semester.

In [15]:
na_counts = ZIP_FEATURES[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                              'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().sum()

na_percentage = ZIP_FEATURES[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                                  'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().mean() * 100


print(na_counts)
print(na_percentage)

LEAID                                  0
earlyfall2020_school_closures      10029
fall2020_school_closures           10024
earlyspring2021_school_closures    16023
spring2021_school_closures         16952
dtype: int64
LEAID                               0.000000
earlyfall2020_school_closures      13.477484
fall2020_school_closures           13.470765
earlyspring2021_school_closures    21.532528
spring2021_school_closures         22.780966
dtype: float64


I tried to consolidate the 2021 semester category for better coverage, but it made no difference

Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [16]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('data/House_Reps_District.csv', dtype=col_types)

In [17]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS',
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)

#ZIP_FEATURES['STATE'] = ZIP_FEATURES.apply(
    #lambda row: row['PHYSICAL STATE'] if pd.isna(row['STATE']) and not pd.isna(row['PHYSICAL STATE']) else row['STATE'], 
    #axis=1
#)

#ZIP_FEATURES = ZIP_FEATURES.drop(columns='PHYSICAL STATE')

ZIP_FEATURES

,ZIP,USPS_STATE,CD,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID,earlyfall2020_school_closures,...,earlyspring2021_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,District,CONGRESSPERSON_PARTY,Years_Served,FIPS_Code,COUNTY_FIPS,TRACT
0,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,0,1,"Neal, Richard E.",Massachusetts,01,Democratic,House: 1989-Present,25,25013,25013813209
1,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,.90322578,1,"McGovern, James P.",Massachusetts,02,Democratic,House: 1997-Present,25,25015,25015820500
2,01002,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,0,1,"McGovern, James P.",Massachusetts,02,Democratic,House: 1997-Present,25,25015,25015820500
3,01004,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,.90322578,1,"McGovern, James P.",Massachusetts,02,Democratic,House: 1997-Present,25,25015,25015820500
4,01004,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,0,1,"McGovern, James P.",Massachusetts,02,Democratic,House: 1997-Present,25,25015,25015820500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75942,99927,AK,0200,0200330,0200700,Hydaburg City School District,Southeast Island School District,Hydaburg City School District,0200330,NaN,...,NaN,NaN,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,02198,02198000100
75943,99927,AK,0200,0200360,0200700,Kake City School District,Southeast Island School District,Kake City School District,0200360,0,...,0,1,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,02198,02198000100
75944,99927,AK,0200,0200700,0200700,Southeast Island School District,Southeast Island School District,Southeast Island School District,0200700,.8897059,...,1,1,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,02198,02198000100
75945,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,1,.56894886,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,02130,02130000100


I have a pretty decent match rate of 95%. However, let's take a look at some of the non-matches and see if we can fill in the gaps

In [18]:
print(ZIP_FEATURES['CONGRESSPERSON'].isna().sum())
print(ZIP_FEATURES['CONGRESSPERSON'].isna().mean() * 100)

525
0.6912715446298077


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits) \
These are cumulative mortality numbers (not weekly)

In [19]:
col_types = {
    'countyFIPS': str
}

usafacts_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/covid_deaths_usafacts.csv', dtype=col_types)
usafacts_deaths['countyFIPS'] = usafacts_deaths['countyFIPS'].astype(str).str.zfill(5)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25. \
I will take covid mortality from the midpoint of the wave range. \
Wave 2 = 6/18-24. 2020 \
Wave 3 = 9/12-20, 2020 \
Wave 4 = 12/1-7, 2020 \
Wave 5 = 2/9-14, 2021 \
Wave 6 = 5/14-25, 2021

In [20]:
columns_to_keep = ['countyFIPS','County Name',
                   '2020-06-01','2020-06-15','2020-06-13','2020-06-20', #w2
                   '2020-09-01','2020-09-16','2020-09-09','2020-09-05', #w3
                   '2020-12-05','2020-12-04','2020-12-01','2020-11-28', #w4
                   '2021-02-12','2021-02-10','2021-02-05','2021-01-29', #w5
                   '2021-05-19','2021-05-17','2021-05-12','2021-05-05'] #w6

usafacts_deaths = usafacts_deaths[columns_to_keep]

new_column_names = [col if idx < 2 else 'COUNTY_COV_DEATHS_' + col.replace('-', '_') for idx, col in enumerate(columns_to_keep)]

usafacts_deaths.columns = new_column_names

usafacts_deaths = usafacts_deaths.rename(columns={
    'countyFIPS': 'COUNTY_FIPS'
})

usafacts_deaths.head()

,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_01,COUNTY_COV_DEATHS_2020_06_15,COUNTY_COV_DEATHS_2020_06_13,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_09_01,COUNTY_COV_DEATHS_2020_09_16,COUNTY_COV_DEATHS_2020_09_09,COUNTY_COV_DEATHS_2020_09_05,...,COUNTY_COV_DEATHS_2020_12_01,COUNTY_COV_DEATHS_2020_11_28,COUNTY_COV_DEATHS_2021_02_12,COUNTY_COV_DEATHS_2021_02_10,COUNTY_COV_DEATHS_2021_02_05,COUNTY_COV_DEATHS_2021_01_29,COUNTY_COV_DEATHS_2021_05_19,COUNTY_COV_DEATHS_2021_05_17,COUNTY_COV_DEATHS_2021_05_12,COUNTY_COV_DEATHS_2021_05_05
0,00000,Statewide Unallocated,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01001,Autauga County,5,6,6,9,24,24,24,24,...,41,41,84,79,76,69,108,108,108,108
2,01003,Baldwin County,9,9,9,9,42,47,46,46,...,137,98,251,239,234,224,310,310,309,307
3,01005,Barbour County,1,1,1,1,7,7,7,7,...,11,10,48,46,44,40,56,56,56,56
4,01007,Bibb County,1,1,1,1,8,9,9,9,...,18,17,57,54,52,51,64,64,64,63


Nearly all ZIP codes get a match (less than 1 percent don't get matched). \
If we end up going back to mortality rates (as of now (01/14/24) we're using prevalence rates. 

In [21]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, usafacts_deaths, on = "COUNTY_FIPS", how = "left")

print(ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_01'].isna().sum())
print(ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_01'].isna().mean() * 100)

483
0.635969821059423


In [22]:
col_types1 = {
    'STATE': str,
    'COUNTY': str
}

col_types2 = {
    'County Code (FIPS)': str,
    'County Subdivision Code (FIPS)': str,
    'Place Code (FIPS)': str,
    'Consolidtated City Code (FIPS)': str,
    'State Code (FIPS)': str
}


census_pop = pd.read_csv('Data/Partisanship and Health Behavior/Data/co-est2021-alldata.csv', encoding='ISO-8859-1',
                        dtype = col_types1)
census_pop_FIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/all-geocodes-v2020.xlsx', 
                                dtype = col_types2)

census_pop_FIPS = census_pop_FIPS.rename(columns={
    'Area Name (including legal/statistical area description)': 'CTYNAME'
})

census_pop['COUNTY_FIPS'] = census_pop['STATE']+census_pop['COUNTY']

State_FIPS_Crosswalk = census_pop[['STATE','STNAME']]
State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop(columns = ['STNAME'])

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop_duplicates(subset='STATE', keep='first')
State_FIPS_Crosswalk = State_FIPS_Crosswalk[pd.notna(State_FIPS_Crosswalk['STATE'])].reset_index(drop=True)

State_FIPS_Crosswalk.to_csv('data/STATE_FIPS_CROSSWALK.csv', index = False)
State_FIPS_Crosswalk.to_csv('data/Partisanship and Health Behavior/Data/STATE_FIPS_CROSSWALK.csv', index = False)

columns_to_keep = ['STATE','POPESTIMATE2020','DEATHS2020','POPESTIMATE2021','DEATHS2021','COUNTY_FIPS']

census_pop = census_pop[columns_to_keep]

census_pop = census_pop.rename(columns={
    'STATE': 'STATE_FIPS'
})

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_89221/4051827355.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_89221/4051827355.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']


A crude mortality rate, not standardized or age adjusted, for this preliminary analysis \
I manually insert Puerto Rico' State FIPS code and District of Columbia to make this a clean merge across the board.

In [23]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, census_pop, on = "COUNTY_FIPS", how = "left")


ZIP_FEATURES['COV_County_CMR_2020_06_20'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_20'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2020_12_09'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_12_05'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2021_05_19'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2021_05_19'] / ZIP_FEATURES['POPESTIMATE2021']) * 1000
ZIP_FEATURES['STATE_FIPS'] = ZIP_FEATURES.apply(lambda row: 72 if row['STATE'] == "Puerto Rico" else row['STATE_FIPS'], axis=1)

print(ZIP_FEATURES['COV_County_CMR_2020_06_20'].isna().sum())
print(ZIP_FEATURES['COV_County_CMR_2020_06_20'].isna().mean() * 100)

491
0.6465034826918772


We have decided that prevalence is a better measure for predicting contact behavior than mortality \
I want these lags \
Wave 2 (6/18-24, 2020), midpoint: 06/21/2020, lag1: 06/14/2020, lag3: 06/07/2020 \
Wave 3 (9/12-20, 2020), midpoint: 09/16/2020, lag1: 09/09/2020, lag3: 09/02/2020 \
Wave 4 (12/1-7, 2020), midpoint: 12/04/2020, lag1: 11/27/2020, lag3: 11/20/2020 \
Wave 5 (2/9-14, 2021), midpoint: 02/11/2021, lag: 02/04/2021, lag3: 01/28/2021 \
Wave 6 (5/14-25, 2021), midpoint: 05/19/2021, lag: 05/12/2021, lag3: 05/05/2021 

In [24]:
col_types = {
    'FIPS': str,
    'iso2':str
}


prevalence_mid_2020 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','6/21/20','6/18/20','6/14/20','6/7/20']

prevalence_mid_2020 = prevalence_mid_2020[prevalence_mid_2020['iso2'] == 'US']
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2020 = prevalence_mid_2020[columns_to_keep]

prevalence_late_2020 =pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_late_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','9/16/20','9/12/20','9/9/20','9/2/20']

prevalence_late_2020 = prevalence_late_2020[prevalence_late_2020['iso2'] == 'US']
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_late_2020 = prevalence_late_2020[columns_to_keep]

prevalence_early_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_early_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','12/4/20','12/1/20','11/27/20','11/20/20']

prevalence_early_2021 = prevalence_early_2021[prevalence_early_2021['iso2'] == 'US']
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_early_2021 = prevalence_early_2021[columns_to_keep]

prevalence_mid_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','2/11/21','2/9/21','2/4/21','1/28/21','5/19/21','5/14/21','5/12/21','5/5/21']

prevalence_mid_2021 = prevalence_mid_2021[prevalence_mid_2021['iso2'] == 'US']
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2021 = prevalence_mid_2021[columns_to_keep]

prevalence = pd.merge(prevalence_mid_2020, prevalence_late_2020, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_early_2021, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_mid_2021, how = "left", on = "FIPS")
prevalence.columns = [col if i == 0 else 'prev_' + col for i, col in enumerate(prevalence.columns)]

prevalence = prevalence.rename(columns={
    'FIPS':'COUNTY_FIPS'
})

prevalence.head()

,COUNTY_FIPS,prev_6/21/20,prev_6/18/20,prev_6/14/20,prev_6/7/20,prev_9/16/20,prev_9/12/20,prev_9/9/20,prev_9/2/20,prev_12/4/20,...,prev_11/27/20,prev_11/20/20,prev_2/11/21,prev_2/9/21,prev_2/4/21,prev_1/28/21,prev_5/19/21,prev_5/14/21,prev_5/12/21,prev_5/5/21
0,01001,428,394,357,265,1619,1432,1398,1345,2945,...,2716,2580,5970,5910,5811,5554,7017,6971,6928,6914
1,01003,415,386,364,313,5003,4752,4639,4458,9501,...,8603,8038,18960,18824,18494,17779,21467,21290,21170,21035
2,01005,271,262,234,193,809,620,618,617,1217,...,1171,1151,2030,2019,1989,1920,2324,2319,2314,2307
3,01007,124,118,104,77,612,576,564,541,1270,...,1173,1024,2377,2362,2327,2271,2652,2630,2612,2604
4,01009,146,127,110,72,1487,1114,1082,1036,3231,...,2888,2704,5955,5929,5842,5612,6808,6750,6680,6635


below we turn these figures into rates based on population

In [25]:
ZIP_FEATURES

,ZIP,USPS_STATE,CD,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID,earlyfall2020_school_closures,...,COUNTY_COV_DEATHS_2021_05_12,COUNTY_COV_DEATHS_2021_05_05,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19
0,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,1504.0,1498.0,25,464908.0,1387.0,462718.0,5619.0,1.391673,1.991792,3.259004
1,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,294.0,293.0,25,162064.0,416.0,161572.0,1537.0,0.654063,1.018116,1.819622
2,01002,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,294.0,293.0,25,162064.0,416.0,161572.0,1537.0,0.654063,1.018116,1.819622
3,01004,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,294.0,293.0,25,162064.0,416.0,161572.0,1537.0,0.654063,1.018116,1.819622
4,01004,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,294.0,293.0,25,162064.0,416.0,161572.0,1537.0,0.654063,1.018116,1.819622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75942,99927,AK,0200,0200330,0200700,Hydaburg City School District,Southeast Island School District,Hydaburg City School District,0200330,NaN,...,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
75943,99927,AK,0200,0200360,0200700,Kake City School District,Southeast Island School District,Kake City School District,0200360,0,...,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
75944,99927,AK,0200,0200700,0200700,Southeast Island School District,Southeast Island School District,Southeast Island School District,0200700,.8897059,...,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
75945,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,1.0,1.0,02,13902.0,26.0,13754.0,106.0,0.000000,0.000000,0.072706


In [26]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, prevalence, on = "COUNTY_FIPS", how = "left")

for col in ZIP_FEATURES.columns:
    # Check if the column name starts with 'prev_'
    if col.startswith('prev_'):
        # Extract the date part of the column name
        date_part = col.split('_', 1)[1]
        # Perform the calculation and create a new column
        ZIP_FEATURES['rate_prev_' + date_part] = (ZIP_FEATURES[col] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
        
prev_columns = prevalence.columns
prev_columns = prev_columns[1:]
ZIP_FEATURES = ZIP_FEATURES.drop(columns=prev_columns)

print(ZIP_FEATURES['rate_prev_11/27/20'].isna().sum())
print(ZIP_FEATURES['rate_prev_11/27/20'].isna().mean() * 100)

491
0.6465034826918772


Below I insert data on urbanicity

In [27]:
columns_to_keep = ['FIPS','RUCC_2013']

col_types = {
    'FIPS': str,
}

RUC_county_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/ruralurbancodes2013.csv', dtype = col_types)

RUC_county_df = RUC_county_df[columns_to_keep]

RUC_county_df = RUC_county_df.rename(columns={
    'RUCC_2013': 'COUNTY_RUCC_2013',
    'FIPS':'COUNTY_FIPS'
})

RUC_county_df['COUNTY_FIPS'] = RUC_county_df['COUNTY_FIPS'].astype(str)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, RUC_county_df, on = "COUNTY_FIPS", how = "left")

Next, I want to control for the severity of the lockdowns according to Oxford's tracker

In [28]:
State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')
State_Restrictions_df['Date'] = State_Restrictions_df['Date'].astype(str)
State_Restrictions_df = State_Restrictions_df[State_Restrictions_df['Date'].isin(['20200617', '20201129'])]
columns_to_keep = ['RegionName','GovernmentResponseIndex_SimpleAverage','Date']
State_Restrictions_df = State_Restrictions_df[columns_to_keep]

State_Restrictions_df = State_Restrictions_df.rename(columns={
    'RegionName': 'STATE',
    'GovernmentResponseIndex_SimpleAverage':'State_Government_Response_Index'
})

State_Restrictions_df

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_89221/750958945.py:1: DtypeWarning: Columns (82,93,121,122) have mixed types. Specify dtype option on import or set low_memory=False.
  State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')


,STATE,State_Government_Response_Index,Date
168,NaN,62.50,20200617
333,NaN,67.19,20201129
534,Alaska,57.81,20200617
699,Alaska,53.65,20201129
900,Alabama,59.64,20200617
...,...,...,...
18267,Wisconsin,53.65,20201129
18468,West Virginia,61.46,20200617
18633,West Virginia,56.25,20201129
18834,Wyoming,59.90,20200617


In [29]:
State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20200617" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_06172020'] == "0" else row['State_Government_Response_Index_06172020'], axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20201129" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_11192020'] == "0" else row['State_Government_Response_Index_11192020'], axis=1
)

State_Restrictions_df

,STATE,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
168,NaN,62.50,20200617,62.50,62.50
333,NaN,67.19,20201129,67.19,67.19
534,Alaska,57.81,20200617,57.81,57.81
699,Alaska,53.65,20201129,53.65,53.65
900,Alabama,59.64,20200617,59.64,59.64
...,...,...,...,...,...
18267,Wisconsin,53.65,20201129,53.65,53.65
18468,West Virginia,61.46,20200617,61.46,61.46
18633,West Virginia,56.25,20201129,56.25,56.25
18834,Wyoming,59.90,20200617,59.90,59.90


In [30]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, State_Restrictions_df, on = "STATE", how = "left")

ZIP_FEATURES

,ZIP,USPS_STATE,CD,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID,earlyfall2020_school_closures,...,rate_prev_1/28/21,rate_prev_5/19/21,rate_prev_5/14/21,rate_prev_5/12/21,rate_prev_5/5/21,COUNTY_RUCC_2013,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
0,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,78.264947,111.630688,111.084344,110.843436,109.819577,2.0,65.10,20200617,65.10,65.10
1,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,78.264947,111.630688,111.084344,110.843436,109.819577,2.0,60.94,20201129,60.94,60.94
2,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,33.912528,56.224701,56.058101,55.959374,55.496594,2.0,65.10,20200617,65.10,65.10
3,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,33.912528,56.224701,56.058101,55.959374,55.496594,2.0,60.94,20201129,60.94,60.94
4,01002,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,33.912528,56.224701,56.058101,55.959374,55.496594,2.0,65.10,20200617,65.10,65.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151889,99927,AK,0200,0200700,0200700,Southeast Island School District,Southeast Island School District,Southeast Island School District,0200700,.8897059,...,13.100437,22.707424,20.960699,19.563319,18.689956,9.0,53.65,20201129,53.65,53.65
151890,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,20.140987,43.303122,40.497770,37.692418,33.160696,7.0,57.81,20200617,57.81,57.81
151891,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,20.140987,43.303122,40.497770,37.692418,33.160696,7.0,53.65,20201129,53.65,53.65
151892,99929,AK,0200,0200810,0200810,Wrangell School District,Wrangell School District,Wrangell School District,0200810,1,...,12.206573,26.760563,26.760563,26.760563,26.291080,9.0,57.81,20200617,57.81,57.81


Next, I want to read in the county level presidential data and generate two variables \
1. For which president did the county overall vote for?
2. What percentage of the county voted for which president?

note: I manually added in DC because it was not in the dataset

In [31]:
col_types = {
    'county_fips': str,
}
county_pres = pd.read_csv('Data/Partisanship and Health Behavior/Data/countypres_2000-2020.csv', dtype = col_types)
county_pres = county_pres[county_pres['year'] == 2020].reset_index(drop=True)

county_pres['party_fips'] = county_pres['party'].astype(str) + county_pres['county_fips'].astype(str)

#there was an issue where there were multiple types of votes on each line for the same person
total_votes_by_party = county_pres.groupby('party_fips')['candidatevotes'].sum().reset_index()
total_votes_by_party.rename(columns={'candidatevotes': 'total_candidatevotes'}, inplace=True)

county_pres = county_pres.merge(total_votes_by_party, on='party_fips', how='left')

county_pres['percentage_won'] = (county_pres['total_candidatevotes'] / county_pres['totalvotes'])*100

# Group by 'countyfips' and find the index of the row with the largest 'percentage_won' in each group
county_pres_wide = county_pres.groupby('county_fips')['percentage_won'].idxmax()

# Select rows from the original DataFrame corresponding to these indices
county_pres_wide = county_pres.loc[county_pres_wide]

county_pres_wide = county_pres_wide.rename(columns={
    'candidate': 'winning_pres_candidate',
    'party':'winning_pres_party',
    'percentage_won': 'winning_pres_candidate_percentage'
})

#merge in the dems
dem_county_pres = county_pres[county_pres['party'] == 'DEMOCRAT']

dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)

dem_county_pres = dem_county_pres[['biden_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(dem_county_pres, on='county_fips', how='left')

#merge in the reps
rep_county_pres = county_pres[county_pres['party'] == 'REPUBLICAN']

rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)

rep_county_pres = rep_county_pres[['trump_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(rep_county_pres, on='county_fips', how='left')

#keeping only the columns I want
county_pres_wide = county_pres_wide.drop(columns =['year','state','state_po','county_name','office','candidatevotes','totalvotes','version','mode','party_fips'])

county_pres_wide

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_89221/345468839.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_89221/345468839.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)


,county_fips,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,01001,DONALD J TRUMP,REPUBLICAN,19838,71.436802,27.018365,71.436802
1,01003,DONALD J TRUMP,REPUBLICAN,83544,76.171373,22.409030,76.171373
2,01005,DONALD J TRUMP,REPUBLICAN,5622,53.451226,45.788173,53.451226
3,01007,DONALD J TRUMP,REPUBLICAN,7525,78.426264,20.698280,78.426264
4,01009,DONALD J TRUMP,REPUBLICAN,24711,89.571553,9.569378,89.571553
...,...,...,...,...,...,...,...
12865,56037,DONALD J TRUMP,REPUBLICAN,12229,73.236316,22.894957,73.236316
12866,56039,JOSEPH R BIDEN JR,DEMOCRAT,9848,66.599040,66.599040,29.356868
12867,56041,DONALD J TRUMP,REPUBLICAN,7496,79.247278,16.819960,79.247278
12868,56043,DONALD J TRUMP,REPUBLICAN,3245,80.481151,16.145833,80.481151


In [32]:
county_pres_wide = county_pres_wide.rename(columns={
    'county_fips': 'COUNTY_FIPS'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, county_pres_wide, on = "COUNTY_FIPS", how = "left")

ZIP_FEATURES

,ZIP,USPS_STATE,CD,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID,earlyfall2020_school_closures,...,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,65.10,20200617,65.10,65.10,JOSEPH R BIDEN JR,DEMOCRAT,125948.0,57.043475,57.043475,39.547449
1,01001,MA,2501,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,0,...,60.94,20201129,60.94,60.94,JOSEPH R BIDEN JR,DEMOCRAT,125948.0,57.043475,57.043475,39.547449
2,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,65.10,20200617,65.10,65.10,JOSEPH R BIDEN JR,DEMOCRAT,63362.0,71.731649,71.731649,25.224154
3,01002,MA,2502,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,0,...,60.94,20201129,60.94,60.94,JOSEPH R BIDEN JR,DEMOCRAT,63362.0,71.731649,71.731649,25.224154
4,01002,MA,2502,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,0,...,65.10,20200617,65.10,65.10,JOSEPH R BIDEN JR,DEMOCRAT,63362.0,71.731649,71.731649,25.224154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423865,99927,AK,0200,0200700,0200700,Southeast Island School District,Southeast Island School District,Southeast Island School District,0200700,.8897059,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
423866,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN
423867,99928,AK,0200,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
423868,99929,AK,0200,0200810,0200810,Wrangell School District,Wrangell School District,Wrangell School District,0200810,1,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN


Next, for the sake of consistency, I want to get data on the percentage of the CD that was won by a democrat and Republican, respectively. 

However, I need a clean crosswalk (as this one is pulling out several duplicates)

In [33]:
col_types = {
    'DISTRICT': str,
}

county_CD_share = pd.read_excel('Data/Partisanship and Health Behavior/Data/federalelections2020.xlsx', 
                                sheet_name='13_US_House_Results_State', 
                                dtype=col_types)

county_CD_share = county_CD_share.rename(columns={
    'DISTRICT': 'CD_DISTRICT'
})

county_CD_share = county_CD_share[pd.notna(county_CD_share['GENERAL %'])].reset_index(drop=True)

county_CD_share = pd.merge(county_CD_share, State_FIPS_Crosswalk, how='left', on="STATE")

county_CD_share['STATE_FIPS'] = county_CD_share['STATE_FIPS'].astype(str)
county_CD_share['CD_DISTRICT'] = county_CD_share['CD_DISTRICT'].astype(str)

county_CD_share['STATE_FIPS'] = county_CD_share['STATE_FIPS'].apply(lambda x: '0' + x if len(x) == 1 else x)
county_CD_share['CD_DISTRICT'] = county_CD_share['CD_DISTRICT'].apply(lambda x: '0' + x if len(x) == 1 else x)

county_CD_share['CD'] = county_CD_share['STATE_FIPS'] + county_CD_share['CD_DISTRICT']

county_CD_share.to_csv('data/CD_COUNTY_PERCENT_WON.csv')

D_county_CD_share = county_CD_share[county_CD_share['PARTY'].str.contains("D")].reset_index(drop=True)
D_county_CD_share = D_county_CD_share[['GENERAL %','CD']]

D_county_CD_share = D_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_DEMOCRAT'
})

R_county_CD_share = county_CD_share[county_CD_share['PARTY'].str.contains("R")].reset_index(drop=True)
R_county_CD_share = R_county_CD_share[['GENERAL %','CD']]

R_county_CD_share = R_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_REPUBLICAN'
})

D_county_CD_share = D_county_CD_share[D_county_CD_share['CD'].notna()]
R_county_CD_share = R_county_CD_share[R_county_CD_share['CD'].notna()]

ZIP_FEATURES = pd.merge(ZIP_FEATURES, D_county_CD_share, how = 'left', on = "CD")
ZIP_FEATURES = pd.merge(ZIP_FEATURES, R_county_CD_share, how = 'left', on = "CD")

print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().sum())
print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().mean() * 100)

#manually adjusting these rows that received 0 votes in democratic counties
ZIP_FEATURES['CD_PERCENT_REPUBLICAN'] = ZIP_FEATURES.apply(
    lambda row: 0 if pd.notna(row['CD_PERCENT_DEMOCRAT']) and pd.isna(row['CD_PERCENT_REPUBLICAN']) else row['CD_PERCENT_REPUBLICAN'],
    axis=1
)

#now in republican counties
ZIP_FEATURES['CD_PERCENT_DEMOCRAT'] = ZIP_FEATURES.apply(
    lambda row: 0 if row['CD_PERCENT_REPUBLICAN'] != 0 and pd.notna(row['CD_PERCENT_REPUBLICAN']) and pd.isna(row['CD_PERCENT_DEMOCRAT']) else row['CD_PERCENT_DEMOCRAT'],
    axis=1
)

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: 0 if row['CD'] == "1225" else row['CD'],
                                                        axis=1)

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: 100 if row['CD'] == "1225" else row['CD'],
                                                        axis=1)

print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().sum())
print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().mean() * 100)

6534
1.3749421320651487
1130
0.23778460502504103
1130
0.23778460502504103


In [34]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv', index = False)

In [35]:
ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/ZIP_Features.csv', index = False)

Why is resp_educ and education different?